# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]


In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:




X=pd.get_dummies(df, columns=['home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 
                                        'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 
                                        'debt_settlement_flag']).drop('loan_status', axis=1)


# check so that the binary data has been created
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [6]:
# Assign the column 'loan_status' to y (our output data-target)
y=df['loan_status'] 
print(y)

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object


In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


# Split the Data into Training and Testing

In [8]:
# Check the balance of our target values
#y.value_counts()
df['loan_status'].value_counts().to_frame()

,loan_status
low_risk,68470
high_risk,347


In [9]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [10]:
print(f'y_train:{Counter(y_train)}')
print(f'y_test: {Counter(y_test)}')

y_train:Counter({'low_risk': 51366, 'high_risk': 246})
y_test: Counter({'low_risk': 17104, 'high_risk': 101})


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(f'y-resampled : {Counter(y_resampled)}')



y-resampled : Counter({'low_risk': 51366, 'high_risk': 51366})


In [12]:
Counter(['loan_status'])

Counter({'loan_status': 1})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [14]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
b1=balanced_accuracy_score(y_test, y_pred)
b1



0.6663237827524566

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, y_pred)
matrix

array([[   71,    30],
       [ 6334, 10770]], dtype=int64)

In [16]:
#Creating a dataframe of the confusion matrix for better visualization

matrix_df = pd.DataFrame(matrix, index=['Actual HighRisk Loans', 'Actual LowRisk Loans'], columns=['PredictTrue', 'PredictFalse'])
matrix_df.style.set_table_styles([{'selector' : '',
                            'props' : [('border',
                                        '5px solid green')]}])

,PredictTrue,PredictFalse
Actual HighRisk Loans,71,30
Actual LowRisk Loans,6334,10770


In [17]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.70      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.70      0.77      0.67      0.44     17205



In [18]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.70      0.02       101
    low_risk       1.00      0.63      0.77     17104

    accuracy                           0.63     17205
   macro avg       0.50      0.67      0.40     17205
weighted avg       0.99      0.63      0.77     17205



In [19]:

NRO_report=metrics.classification_report(y_test, y_pred, output_dict=True)
NRO_report_df = pd.DataFrame(NRO_report).transpose()
NRO_report_df.round(2)


,precision,recall,f1-score,support
high_risk,0.01,0.70,0.02,101.00
low_risk,1.00,0.63,0.77,17104.00
accuracy,0.63,0.63,0.63,0.63
macro avg,0.50,0.67,0.40,17205.00
weighted avg,0.99,0.63,0.77,17205.00


### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
print(Counter(y_resampled))
print(Counter(['loan_status']))

Counter({'low_risk': 51366, 'high_risk': 51366})
Counter({'loan_status': 1})


In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [22]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
b2 = balanced_accuracy_score(y_test, y_pred)
b2

0.6623064259185507

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix


matrix2=confusion_matrix(y_test, y_pred)
matrix2

array([[   64,    37],
       [ 5286, 11818]], dtype=int64)

In [24]:
# create a dataframe for visualization
matrix2_df=pd.DataFrame(matrix2, index=['Actual HighRisk Loan', 'Actual LowRisk Loan'], columns=['PredictTrue', 'PredictFalse'])
# making an orange border
matrix2_df.style.set_table_styles([{'selector' : '',
                            'props' : [('border',
                                        '5px solid green')]}])

,PredictTrue,PredictFalse
Actual HighRisk Loan,64,37
Actual LowRisk Loan,5286,11818


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.69      0.02      0.66      0.44       101
   low_risk       1.00      0.69      0.63      0.82      0.66      0.44     17104

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



In [26]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.63      0.02       101
    low_risk       1.00      0.69      0.82     17104

    accuracy                           0.69     17205
   macro avg       0.50      0.66      0.42     17205
weighted avg       0.99      0.69      0.81     17205



In [27]:
SMOTE_report=metrics.classification_report(y_test, y_pred, output_dict=True)
SMOTE_report_df = pd.DataFrame(SMOTE_report).transpose()
SMOTE_report_df.round(2)


,precision,recall,f1-score,support
high_risk,0.01,0.63,0.02,101.00
low_risk,1.00,0.69,0.82,17104.00
accuracy,0.69,0.69,0.69,0.69
macro avg,0.50,0.66,0.42,17205.00
weighted avg,0.99,0.69,0.81,17205.00


# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [30]:
Counter(['loan_status'])

Counter({'loan_status': 1})

In [31]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
matrix3= confusion_matrix(y_test, y_pred)
matrix3

array([[   70,    31],
       [10340,  6764]], dtype=int64)

In [33]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
b3 = balanced_accuracy_score(y_test, y_pred)
b3

0.5442661782548694

In [34]:
# create a dataframe for better visualization:
matrix3_df=pd.DataFrame(matrix3, index=['Act. HighRisk Loans', 'Act. LowRisk Loans'], columns=['PredictTrue', 'PredictFalse'])

matrix3_df.style.set_table_styles([{'selector' : '',
                            'props' : [('border',
                                        '5px solid yellow')]}])

,PredictTrue,PredictFalse
Act. HighRisk Loans,70,31
Act. LowRisk Loans,10340,6764


In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.40      0.01      0.52      0.28       101
   low_risk       1.00      0.40      0.69      0.57      0.52      0.27     17104

avg / total       0.99      0.40      0.69      0.56      0.52      0.27     17205



In [36]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.69      0.01       101
    low_risk       1.00      0.40      0.57     17104

    accuracy                           0.40     17205
   macro avg       0.50      0.54      0.29     17205
weighted avg       0.99      0.40      0.56     17205



In [37]:
cc_report=metrics.classification_report(y_test, y_pred, output_dict=True)
cc_report_df = pd.DataFrame(cc_report).transpose()
cc_report_df.round(2)

,precision,recall,f1-score,support
high_risk,0.01,0.69,0.01,101.0
low_risk,1.00,0.40,0.57,17104.0
accuracy,0.40,0.40,0.40,0.4
macro avg,0.50,0.54,0.29,17205.0
weighted avg,0.99,0.40,0.56,17205.0


# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [38]:
# Use the SMOTEENN technique to perform combination sampling on the data
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
print(Counter(y_resampled))
print(Counter(['loan_status']))

Counter({'high_risk': 68458, 'low_risk': 62022})
Counter({'loan_status': 1})


In [40]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [41]:

# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
matrix4= confusion_matrix(y_test, y_pred)
matrix4

array([[  78,   23],
       [7187, 9917]], dtype=int64)

In [43]:


# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

b4 = balanced_accuracy_score(y_test, y_pred)
b4

0.6760415605405256

In [44]:
# create a dataframe for better visualization:
matrix4_df=pd.DataFrame(matrix4, index=['Actual HighRisk Loans', 'Actual LowRisk Loans'], columns=['PredictTrue', 'PredictFalse'])
matrix4_df.style.set_table_styles([{'selector' : '',
                            'props' : [('border',
                                        '5px solid orange')]}])

,PredictTrue,PredictFalse
Actual HighRisk Loans,78,23
Actual LowRisk Loans,7187,9917


In [45]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.58      0.02      0.67      0.46       101
   low_risk       1.00      0.58      0.77      0.73      0.67      0.44     17104

avg / total       0.99      0.58      0.77      0.73      0.67      0.44     17205



In [46]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.77      0.02       101
    low_risk       1.00      0.58      0.73     17104

    accuracy                           0.58     17205
   macro avg       0.50      0.68      0.38     17205
weighted avg       0.99      0.58      0.73     17205



In [47]:
combi_report=metrics.classification_report(y_test, y_pred, output_dict=True)
combi_report_df = pd.DataFrame(combi_report).transpose()
combi_report_df.round(2)

,precision,recall,f1-score,support
high_risk,0.01,0.77,0.02,101.00
low_risk,1.00,0.58,0.73,17104.00
accuracy,0.58,0.58,0.58,0.58
macro avg,0.50,0.68,0.38,17205.00
weighted avg,0.99,0.58,0.73,17205.00


In [48]:
f1score= (NRO_report_df.iat[0,2], SMOTE_report_df.iat[0,2], cc_report_df.iat[0,2],
combi_report_df.iat[0,2])
print(f1score)
type(f1score)

(0.021826006762988015, 0.02348192992111539, 0.013319379697459803, 0.021178387184360575)


tuple

In [49]:
recall = (NRO_report_df.recall['high_risk'], SMOTE_report_df.recall['high_risk'],
cc_report_df.recall['high_risk'], combi_report_df.recall['high_risk'])
print(recall)
type(recall)

(0.7029702970297029, 0.6336633663366337, 0.693069306930693, 0.7722772277227723)


tuple

In [51]:
recall=np.round(recall, 2)
recall

array([0.7 , 0.63, 0.69, 0.77])

In [52]:
precision = (NRO_report_df.precision['high_risk'], SMOTE_report_df.precision['high_risk'], cc_report_df.precision['high_risk'],
combi_report_df.precision['high_risk'])
print(precision)
type(precision)

(0.011085089773614365, 0.011962616822429906, 0.0067243035542747355, 0.010736407432897454)


tuple

In [53]:
precision=np.round(precision,2)
precision

array([0.01, 0.01, 0.01, 0.01])

In [54]:
acc_score=[b1,b2,b3,b4]
acc_score
type(acc_score)

list

In [55]:
acc_score= np.round(acc_score, 2)
acc_score

array([0.67, 0.66, 0.54, 0.68])

In [56]:
# Summarize the results in a summary table

#create rows
rows=['OverSampling-Naive Random', 'Oversampling-SMOTE', 'UnderSampling-ClusterCentroids', 'Combination Sampling']

#create columns and assign values
col={'Balance Accuracy Score': acc_score, 'Precision': precision, 'Sensitivity': recall, 'f1-score': f1score}
    
df_summary= pd.DataFrame(col, rows)
    

df_summary




,Balance Accuracy Score,Precision,Sensitivity,f1-score
OverSampling-Naive Random,0.67,0.01,0.70,0.021826
Oversampling-SMOTE,0.66,0.01,0.63,0.023482
UnderSampling-ClusterCentroids,0.54,0.01,0.69,0.013319
Combination Sampling,0.68,0.01,0.77,0.021178
